### Difference Features

In [1]:
# !sudo apt update
# !sudo apt install openjdk-17-jre-headless -y
import pyspark
# from pyspark import pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import col, substring
from pyspark.sql.types import StructType, StructField, \
StringType, IntegerType, TimestampType, DateType, FloatType

import os

In [2]:
conf = pyspark.SparkConf().setAll([\
            ('spark.app.name', 'Glucose_Analysis_Spark')])\
            .set('spark.sql.shuffle.partitions', '1500')
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()  

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/24 03:23:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.options(header='True', inferSchema='True', delimiter=',')\
        .csv('/cephfs/data/cohort.csv')\
        .withColumnRenamed('_c0', 'NumId')

In [4]:
df.show(2)

23/05/24 03:23:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+-----+--------------------+------+-------------------+---+------------+---------+
|NumId|              UserId|Gender|                DOB|Age|DiabetesType|Treatment|
+-----+--------------------+------+-------------------+---+------------+---------+
|    0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|
|    1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|
+-----+--------------------+------+-------------------+---+------------+---------+
only showing top 2 rows



In [5]:
df = df.withColumn('AgeGroup', substring(df.Age.cast(StringType()), 0,1) * 10)
df = df.withColumn('AgeGroup', df.AgeGroup.cast(IntegerType()))

In [6]:
df = df.withColumnRenamed('Gender', 'Sex')

In [7]:
df.show(3)

23/05/24 03:23:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+-----+--------------------+------+-------------------+---+------------+---------+--------+
|NumId|              UserId|   Sex|                DOB|Age|DiabetesType|Treatment|AgeGroup|
+-----+--------------------+------+-------------------+---+------------+---------+--------+
|    0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|      90|
|    1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|      80|
|    2|uhsyLhr4Zl6NfGbNB...|Female|1938-01-01 00:00:00| 85|    type-two|       no|      80|
+-----+--------------------+------+-------------------+---+------------+---------+--------+
only showing top 3 rows



In [8]:
encodedCols = ['Sex', 'Treatment', 'AgeGroup'] # not doing'DiabetesType' because all type-two
encodedLabels = []

for name in encodedCols:
    indexer = StringIndexer(inputCol=name, outputCol= name + '_Num')
    indexer_fitted = indexer.fit(df)
    encodedLabels.append([name, indexer_fitted.labels])                    
    df = indexer_fitted.transform(df)

In [9]:
encodedLabels

[['Sex', ['Female', 'Male']],
 ['Treatment', ['yes-both', 'yes-long-acting', 'no', 'yes-fast-acting']],
 ['AgeGroup', ['50', '60', '70', '40', '30', '80', '90', '10']]]

One Hot Encoding Way

In [ ]:
single_col_ohe = OneHotEncoder(inputCol="Sex_Num", outputCol="Sex_Encoded", dropLast=True)
df = single_col_ohe.fit(df).transform(df)

single_col_ohe = OneHotEncoder(inputCol="Treatment_Num", outputCol="Treatment_Encoded", dropLast=True)
df = single_col_ohe.fit(df).transform(df)

single_col_ohe = OneHotEncoder(inputCol="AgeGroup_Num", outputCol="AgeGroup_Encoded", dropLast=True)
df = single_col_ohe.fit(df).transform(df)

In [ ]:
df = df.drop('UserId', 'Sex', 'DOB', 'Age', 'DiabetesType', 'Treatment', 'AgeGroup', 'Treatment_Num', \
            'Sex_Num', 'AgeGroup_Num')

In [ ]:
df.show(5)

In [ ]:
df.write.mode('overwrite').parquet('/cephfs/data/cohort_encoded.parquet')

Bool Columns

In [10]:
for label in encodedLabels:
    main = label[0]
    categories = label[1]
    for cat in categories:
        cat = cat.replace('-', '_')
        if main == 'Sex':
            df = df.withColumn(main + '_' + cat, (df.Sex == cat).cast('integer'))
        elif main == 'Treatment':
            df = df.withColumn(main + '_' + cat, (df.Treatment == cat).cast('integer'))
        elif main == 'AgeGroup':
            df = df.withColumn(main + '_' + cat, (df.AgeGroup == cat).cast('integer'))
            

In [11]:
df.columns

['NumId',
 'UserId',
 'Sex',
 'DOB',
 'Age',
 'DiabetesType',
 'Treatment',
 'AgeGroup',
 'Sex_Num',
 'Treatment_Num',
 'AgeGroup_Num',
 'Sex_Female',
 'Sex_Male',
 'Treatment_yes_both',
 'Treatment_yes_long_acting',
 'Treatment_no',
 'Treatment_yes_fast_acting',
 'AgeGroup_50',
 'AgeGroup_60',
 'AgeGroup_70',
 'AgeGroup_40',
 'AgeGroup_30',
 'AgeGroup_80',
 'AgeGroup_90',
 'AgeGroup_10']

In [15]:
df = df.drop('UserId', 'Sex', 'DOB', 'Age', 'DiabetesType', 'Treatment', 'AgeGroup', 'Treatment_Num', \
            'Sex_Num', 'AgeGroup_Num')

In [16]:
df.dtypes

[('NumId', 'int'),
 ('Sex_Female', 'int'),
 ('Sex_Male', 'int'),
 ('Treatment_yes_both', 'int'),
 ('Treatment_yes_long_acting', 'int'),
 ('Treatment_no', 'int'),
 ('Treatment_yes_fast_acting', 'int'),
 ('AgeGroup_50', 'int'),
 ('AgeGroup_60', 'int'),
 ('AgeGroup_70', 'int'),
 ('AgeGroup_40', 'int'),
 ('AgeGroup_30', 'int'),
 ('AgeGroup_80', 'int'),
 ('AgeGroup_90', 'int'),
 ('AgeGroup_10', 'int')]

In [18]:
df.write.mode('overwrite').parquet('/cephfs/data/cohort_bool_encoded.parquet')

23/05/24 03:24:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Gender, Age, Treatment
 Schema: _c0, Gender, Age, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv


---------------------------

In [31]:
types = ['train', 'test', 'val']

for dataType in types:
    
    files_directory=os.listdir('/cephfs/summary_stats/' + dataType + '_cat')
    files=['/cephfs/summary_stats/' + dataType + '_cat/' + i for i in files_directory if not ('.crc' in i or 'SUCCESS' in i)]

    # Read in Summary Statistics
    summary_stats= spark.read.format('parquet').load(files)
    
    for encodeType in ['cohort_encoded', 'cohort_bool_encoded']:
        one_hot_encoding = spark.read.format('parquet').load('/cephfs/data/' + encodeType + '.parquet')
        merged = summary_stats.join(one_hot_encoding, on='NumId', how='left')
        
        merged.write.parquet('/cephfs/summary_stats/encoded/one_hot_' + dataType \
                             + '/summary_stats_' + encodeType + '.parquet')        
        

In [24]:
# one_hot_encoding = spark.read.format('parquet').load('/cephfs/data/cohort_encoded.parquet')


In [25]:
import time

start = time.time()
print(time.time()-start)

0.0142364501953125


In [28]:
merged.filter(col('NumId') == 19).select(col('Sex_Female'), col('Sex_Male'),\
                                         col('Treatment_yes_both'), col('AgeGroup_50'),\
                                         col('AgeGroup_60'), col('AgeGroup_70'),\
                                         col('AgeGroup_40'), col('AgeGroup_30'), col('AgeGroup_80'),\
                                         col('AgeGroup_90'), col('AgeGroup_10')).show(5)

+----------+--------+------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|Sex_Female|Sex_Male|Treatment_yes_both|AgeGroup_50|AgeGroup_60|AgeGroup_70|AgeGroup_40|AgeGroup_30|AgeGroup_80|AgeGroup_90|AgeGroup_10|
+----------+--------+------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|         1|       0|                 0|          0|          0|          1|          0|          0|          0|          0|          0|
|         1|       0|                 0|          0|          0|          1|          0|          0|          0|          0|          0|
|         1|       0|                 0|          0|          0|          1|          0|          0|          0|          0|          0|
|         1|       0|                 0|          0|          0|          1|          0|          0|          0|          0|          0|
|         1|       0|                 0| 

In [ ]:
import pathlib

allPaths = [str(x) for x in list(pathlib.Path("/cephfs/train_test_val/train_set").glob('*.parquet')) if 'part-00' in str(x)]


In [ ]:
allPaths[42]

In [ ]:
test = spark.read.format('parquet').load(allPaths[42])

In [ ]:
test.select(col('NumId')).distinct().count()